# Websocket streaming test

Source: https://microsoft.github.io/autogen/docs/notebooks/agentchat_websockets/

## Load env and set up configs

### Load envs

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

### LLM configs

Use these in AutoGen's `llm_config = { "config_list": [...]}`

In [6]:
import os
config_gpt_4o = {
    "model": os.environ.get("GPT_4O_MODEL"),
    "api_type": "azure",
    "api_key": os.environ.get("GPT_4O_API_KEY"),
    "base_url": os.environ.get("GPT_4O_AZURE_ENDPOINT"),
    "api_version": os.environ.get("GPT_4O_API_VERSION"),
}


config_gpt_4_turbo = {
    "model": os.environ.get("GPT_4_TURBO_MODEL"),
    "api_type": "azure",
    "api_key": os.environ.get("GPT_4_TURBO_API_KEY"),
    "base_url": os.environ.get("GPT_4_TURBO_AZURE_ENDPOINT"),
    "api_version": os.environ.get("GPT_4_TURBO_API_VERSION"),
}


config_gpt_35_turbo = {
    "model": os.environ.get("GPT_35_TURBO_MODEL"),
    "api_type": "azure",
    "api_key": os.environ.get("GPT_35_TURBO_API_KEY"),
    "base_url": os.environ.get("GPT_35_TURBO_AZURE_ENDPOINT"),
    "api_version": os.environ.get("GPT_35_TURBO_API_VERSION"),
}

## Test with a Websocket Server

### Initial Connection Handler

For each new client connection utilizing websockets, the websocket server automatically initiates a new instance of `IOWebsockets` class for managing client-server data flow. It will be passed to the handler as the `iostream` parameter.

Utilize this to initialize everything to manage the interactive session with AutoGen:
- get initial input message
- instantiate the `UserProxyAgent` and other agents, also register functions the agents need to use
- initiate chat with the `UserProxyAgent`

#### Import dependenies

In [7]:
# Import dependencies
from datetime import datetime
from tempfile import TemporaryDirectory

from websockets.sync.client import connect as ws_connect

import autogen
from autogen.io.websockets import IOWebsockets

#### Define handler function

In [44]:
def on_connect(iostream: IOWebsockets) -> None:
    print(
        f" - on_connect(): Connected to client using IOWebsockets {iostream}",
        flush=True,
    )
    print(" - on_connect(): Receiving a message to the client", flush=True)

    # Get the initial message from the client
    initial_msg = iostream.input()

    # Define a chatbot ConversableAgent that can use weather_forcast function
    agent = autogen.ConversableAgent(
        name="chatbot",
        system_message="Complete a task given to you and reply TERMINATE when the task is done. If asked about the weather, use tool 'weather_forecast(city)' to get the weather forecast for a city.",
        # system_message="Complete a tasks given to you. If asked about the weather, use tool 'weather_forecast(city)' to get the weather forecast for a city.",
        llm_config={
            "config_list": [config_gpt_4o, config_gpt_4_turbo, config_gpt_35_turbo],
            "stream": True,
        },
    )

    # Define code executor using UserProxyAgent
    user_proxy = autogen.UserProxyAgent(
        name="user_proxy",
        system_message="A proxy for the user",
        # is_termination_msg=False,
        is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
        human_input_mode="ALWAYS",
        max_consecutive_auto_reply=10,
        code_execution_config=False,
    )

    # Functions that agent can use
    def weather_forecast(city: str) -> str:
        return f"The weather forcast for {city} at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} is 25°C and sunny."

    autogen.register_function(
        weather_forecast,
        caller=agent,
        executor=user_proxy,
        description="Weather forecast for a city",
    )

    # Initiate conversation
    print(
        f" - on_connect(): Initiating chat with agent {agent} using message {initial_msg}",
        flush=True,
    )
    user_proxy.initiate_chat(agent, message=initial_msg)

## Test websocket server with a Python client

In [13]:
with IOWebsockets.run_server_in_thread(on_connect=on_connect, port=8765) as uri:
    print(f" - test_setup() with websocket server running on {uri}", flush=True)
    
    with ws_connect(uri) as websocket:
        print(f" - Connected to server on {uri}", flush=True)
        print(f" - Sending message to server", flush=True)
        websocket.send("Check current weather in Tokyo and write a Haiku about it.")
        
        while True:
            message = websocket.recv()
            message = message.decode("utf-8") if isinstance(message, bytes) else message
            
            print(message, end="", flush=True)
            
            if "TERMINATE" in message:
                print()
                print(" - Received TERMINATE message. Exiting...", flush=True)
                break

 - test_setup() with websocket server running on ws://127.0.0.1:8765
 - on_connect(): Connected to client using IOWebsockets <autogen.io.websockets.IOWebsockets object at 0x7f0d99959760>
 - on_connect(): Receiving a message to the client
 - Connected to server on ws://127.0.0.1:8765
 - Sending message to server
 - on_connect(): Initiating chat with agent <autogen.agentchat.conversable_agent.ConversableAgent object at 0x7f0de40b29c0> using message Check current weather in Tokyo and write a Haiku about it.
user_proxy (to chatbot):

Check current weather in Tokyo and write a Haiku about it.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
chatbot (to user_proxy):


***** Suggested tool call (call_ba3qVvqFWRT4Ox5pjAF2VObk): weather_forecast *****
Arguments: 
{"city": "Tokyo"}
*********************************************************************************

-----------------------------------------------------------------------

## Test websocket server with a frontend and app server

- HTML Template - frontend UI for user interaction
- FastAPI application - app server to serve HTML frontend
- Websocket server - stream interaction with AutoGen

#### Import dependencies

In [14]:
from contextlib import asynccontextmanager
from pathlib import Path
from fastapi import FastAPI
from fastapi.responses import HTMLResponse

#### Frontend HTML and JS

In [45]:
html = """
<!DOCTYPE html>
<html>
    <head>
        <title>Autogen websocket test</title>
    </head>
    <body>
        <h1>WebSocket Chat</h1>
        <form action="" onsubmit="sendMessage(event)">
            <input type="text" id="messageText" autocomplete="off"/>
            <button>Send</button>
        </form>
        <ul id='messages'>
        </ul>
        <script>
            var ws = new WebSocket("ws://localhost:8080/ws");
            ws.onmessage = function(event) {
                var messages = document.getElementById('messages')
                var message = document.createElement('li')
                var content = document.createTextNode(event.data)
                message.appendChild(content)
                messages.appendChild(message)
            };
            function sendMessage(event) {
                event.preventDefault()
                var input = document.getElementById("messageText")
                console.log("Will send: ", input.value)
                ws.send(input.value)
                input.value = ''
            }
        </script>
    </body>
</html>
"""

#### FastAPI App 

In [46]:
@asynccontextmanager
async def run_websocket_server(app):
    with IOWebsockets.run_server_in_thread(on_connect=on_connect, port=8080) as uri:
        print(f"Websocket server started at {uri}.", flush=True)
        yield

app = FastAPI(lifespan=run_websocket_server)

@app.get("/")
async def get():
    return HTMLResponse(html)

#### Use `uvicorn` to run the server

In [47]:
import uvicorn

config = uvicorn.Config(app)
server = uvicorn.Server(config)
await server.serve()

INFO:     Started server process [324416]
INFO:     Waiting for application startup.


Websocket server started at ws://127.0.0.1:8080.


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:36142 - "GET / HTTP/1.1" 200 OK
 - on_connect(): Connected to client using IOWebsockets <autogen.io.websockets.IOWebsockets object at 0x7f0d959b2210>
 - on_connect(): Receiving a message to the client
 - on_connect(): Initiating chat with agent <autogen.agentchat.conversable_agent.ConversableAgent object at 0x7f0d9610ed80> using message Hello, how are you?


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [324416]
